In [29]:
import cv2

In [30]:
from deepface import DeepFace
import numpy as np

In [31]:
import tensorflow as tf

In [32]:
import datetime,pygame,keyboard

In [33]:
import xlwings as xw

In [38]:
#xlwings 
workbook = xw.Book('attendance.xlsx')
sheet_name = datetime.date.today().isoformat()
try:
    worksheet = workbook.sheets(sheet_name)
except:
    worksheet = workbook.sheets.add(sheet_name)

In [39]:
worksheet.range('A1').value = 'STUDENT'
worksheet.range('B1').value = 'DATE'
worksheet.range('C1').value = 'TIME'
students=[]

In [40]:
model_live = 'liveness.model'
model_live = tf.keras.models.load_model(model_live)

In [41]:
s=2
i=0
entered = False
al_entered = False

In [42]:
cap = cv2.VideoCapture(0)

#pygame
pygame.init()
pg_txt = (39,112,50)
pg_bg = (154,179,157)
font = cv2.FONT_HERSHEY_SIMPLEX
pygame_font = pygame.font.Font('freesansbold.ttf',32)

screen = pygame.display.set_mode((960,540))

bgimg = pygame.image.load('bg_img.png')
bgimg = pygame.transform.rotozoom(bgimg,0,1)

screen.blit(bgimg,(0,0))

#day today 
t0 = datetime.date.today()
t0 = t0.day

running = True

while running:
    moment = datetime.datetime.now()
    hr = moment.hour 
    mins = moment.minute 
    day = moment.day
    month = moment.month
    year = moment.year
    date = f"{day}-{month}-{year}"
    time = f"{hr}:{mins}"
        
    screen.blit(bgimg,(0,0))
        
    #check 
    
    if day!=t0:
        t0 = day
        worksheet = workbook.sheets.add(date)
        worksheet.range('A1').value = 'STUDENT'
        worksheet.range('B1').value = 'DATE'
        worksheet.range('C1').value = 'TIME'
        students=[]
        i=0
        entered =  False
        al_entered = False
        
    state,frame = cap.read()
    
    if state!=True:
        break 
    res = DeepFace.find(frame, db_path='./Deepface', enforce_detection = False, model_name='VGG-Face')
    if (res[0]['VGG-Face_cosine'][0])<(0.19):  
        if len(res[0]['identity'][0])>0:
            name = res[0]['identity'][0].split('/')[1].split('\\')[1]
            val = res[0]['VGG-Face_cosine'][0]
            xmin = res[0]['source_x'][0]
            ymin = res[0]['source_y'][0]
            w = res[0]['source_w'][0]
            h = res[0]['source_h'][0]
            xmax = xmin+w
            ymax = ymin+h

            #liveness

            img = frame[ymin:ymax,xmin:xmax]
            img = cv2.resize(img,(32,32))
            # model accepts the liveness to (32,32) instead of the original one it will resize
            img = img.astype('float')/255.0 
            # to normalize we divide as all values in rgb range from 0-255 we are now converting to 0-1
            img = tf.keras.preprocessing.image.img_to_array(img)
            img = np.expand_dims(img,axis=0)

            liveness = model_live.predict(img)

            liveness = liveness[0].argmax()

            if liveness == 1:
                cv2.rectangle(frame, (xmin,ymin), (xmax,ymax), (0,255,0),2)
                cv2.putText(frame,name,(xmin,ymin),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3,cv2.LINE_AA)
                cv2.putText(frame, str(liveness), (xmin,ymax+25),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2,cv2.LINE_AA )


                n_txt = pygame_font.render(name,True,pg_txt,pg_bg)
                n_textrect = n_txt.get_rect()
                n_textrect.center = (960-200,540//2)
                
                
                d_txt = pygame_font.render(date,True,pg_txt,pg_bg)
                d_textrect = d_txt.get_rect()
                d_textrect.center = (960-200,(540//2)+40)

                t_txt = pygame_font.render(time,True,pg_txt,pg_bg)
                t_textrect = t_txt.get_rect()
                t_textrect.center = (960-200,(540//2)+(80))
                
                v_txt = pygame_font.render(str(val),True,pg_txt,pg_bg)
                v_textrect = v_txt.get_rect()
                v_textrect.center = (960-200,(540//2)+(120))


                screen.blit(n_txt, n_textrect)
                screen.blit(d_txt, d_textrect)
                screen.blit(t_txt, t_textrect)
                screen.blit(v_txt, v_textrect)

                if keyboard.is_pressed('enter'):
                    if name not in students:
                        worksheet.range(f'A{s}').value = name
                        worksheet.range(f'B{s}').value = date
                        worksheet.range(f'C{s}').value = time
                        students.append(name)
                        s+=1
                        entered = True
                    else:
                        al_entered = True

                if i<1 and entered==True:
                    e_txt = pygame_font.render('entered',True,pg_txt,pg_bg)
                    e_textrect = e_txt.get_rect()
                    e_textrect.center = (960-200,(540//2)+120)
                    screen.blit(e_txt,e_textrect)
                    i+=1
                elif i<30 and al_entered==True:
                    ae_txt = pygame_font.render('already entered',True,pg_txt,pg_bg)
                    ae_textrect = ae_txt.get_rect()
                    ae_textrect.center = (960-200,(540//2)+120)
                    screen.blit(ae_txt,ae_textrect)
                    i+=1
                else:
                    entered = False
                    al_entered = False
                    i=0
                    
    elif (res[0]['VGG-Face_cosine'].empty):
        xmin = res[0]['source_x'][0]
        ymin = res[0]['source_y'][0]
        val = res[0]['VGG-Face_cosine'][0]
        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]
        xmax = xmin+w
        ymax = ymin+h
        img = frame[ymin:ymax,xmin:xmax]
        img = cv2.resize(img,(32,32))
        # model accepts the liveness to (32,32) instead of the original one it will resize
        img = img.astype('float')/255.0 
        # to normalize we divide as all values in rgb range from 0-255 we are now converting to 0-1
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = np.expand_dims(img,axis=0)

        liveness = model_live.predict(img)

        liveness = liveness[0].argmax()

        if liveness == 1:
            cv2.rectangle(frame, (xmin,ymin), (xmax,ymax), (0,255,0),2)
            cv2.putText(frame, str(liveness), (xmin,ymax+25),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2,cv2.LINE_AA )


            n_txt = pygame_font.render('NOT FOUND',True,pg_txt,pg_bg)
            n_textrect = n_txt.get_rect()
            n_textrect.center = (960-200,540//2)

            v_txt = pygame_font.render(str(val),True,pg_txt,pg_bg)
            v_textrect = v_txt.get_rect()
            v_textrect.center = (960-200,(540//2)+(120))


            screen.blit(n_txt, n_textrect)
            screen.blit(v_txt, v_textrect)

        
        
    else:
        xmin = res[0]['source_x'][0]
        ymin = res[0]['source_y'][0]
        val = res[0]['VGG-Face_cosine'][0]
        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]
        xmax = xmin+w
        ymax = ymin+h
        img = frame[ymin:ymax,xmin:xmax]
        img = cv2.resize(img,(32,32))
        # model accepts the liveness to (32,32) instead of the original one it will resize
        img = img.astype('float')/255.0 
        # to normalize we divide as all values in rgb range from 0-255 we are now converting to 0-1
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = np.expand_dims(img,axis=0)

        liveness = model_live.predict(img)

        liveness = liveness[0].argmax()

        if liveness == 1:
            cv2.rectangle(frame, (xmin,ymin), (xmax,ymax), (0,255,0),2)
            cv2.putText(frame, str(liveness), (xmin,ymax+25),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2,cv2.LINE_AA )


            n_txt = pygame_font.render('NOT FOUND',True,pg_txt,pg_bg)
            n_textrect = n_txt.get_rect()
            n_textrect.center = (960-200,540//2)

            v_txt = pygame_font.render(str(val),True,pg_txt,pg_bg)
            v_textrect = v_txt.get_rect()
            v_textrect.center = (960-200,(540//2)+(120))


            screen.blit(n_txt, n_textrect)
            screen.blit(v_txt, v_textrect)

        
            
            
        
 
    image = pygame.image.frombuffer(frame.tostring(), frame.shape[1::-1],'BGR')
    image = pygame.transform.rotozoom(image,0,0.8)

    size = image.get_size()
    rect_img = pygame.Surface(size, pygame.SRCALPHA)
    pygame.draw.rect(rect_img,(255,255,255),(0,0,*size), border_radius=10)

    image = image.copy().convert_alpha()
    image.blit(rect_img, (0,0), None, pygame.BLEND_RGB_MIN)
    screen.blit(image,(20,80))

    pygame.display.update()
    for event in  pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        
#     cv2.imshow('attendance', frame)
#     c = cv2.waitKey(1)
#     if c==ord('q'):
#         break 


cap.release()
pygame.quit()
#cv2.destroyAllWindows()

There are  22  representations found in  representations_vgg_face.pkl
find function lasts  0.17377114295959473  seconds
1/1 [==============================] - 0s 116ms/step
There are  22  representations found in  representations_vgg_face.pkl
find function lasts  0.17615318298339844  seconds
1/1 [==============================] - 0s 16ms/step
There are  22  representations found in  representations_vgg_face.pkl
find function lasts  0.1716001033782959  seconds
1/1 [==============================] - 0s 16ms/step
There are  22  representations found in  representations_vgg_face.pkl
find function lasts  0.1747119426727295  seconds
1/1 [==============================] - 0s 17ms/step
There are  22  representations found in  representations_vgg_face.pkl
find function lasts  0.1745283603668213  seconds
1/1 [==============================] - 0s 16ms/step
There are  22  representations found in  representations_vgg_face.pkl
find function lasts  0.17192745208740234  seconds
1/1 [=================

find function lasts  0.17951059341430664  seconds
1/1 [==============================] - 0s 18ms/step
There are  22  representations found in  representations_vgg_face.pkl
find function lasts  0.17807745933532715  seconds
1/1 [==============================] - 0s 16ms/step
There are  22  representations found in  representations_vgg_face.pkl
find function lasts  0.17664122581481934  seconds
1/1 [==============================] - 0s 17ms/step


In [43]:
model = DeepFace.find(img_path = 'testg.jpg', db_path = './Deepface', enforce_detection = False, model_name='VGG-Face')

There are  22  representations found in  representations_vgg_face.pkl
find function lasts  0.2410435676574707  seconds


In [36]:
model[0]

,identity,source_x,source_y,source_w,source_h,VGG-Face_cosine
0,./Deepface\Ganesh/WhatsApp Image 2023-09-15 at...,0,0,576,1024,0.113137
1,./Deepface\Ganesh/WhatsApp Image 2023-09-15 at...,0,0,576,1024,0.135928
2,./Deepface\Akhila/WhatsApp Image 2023-10-15 at...,0,0,576,1024,0.313153
3,./Deepface\Akhila/WhatsApp Image 2023-10-15 at...,0,0,576,1024,0.331407
4,./Deepface\Harshita/WhatsApp Image 2023-09-15 ...,0,0,576,1024,0.335993
5,./Deepface\Akhila/WhatsApp Image 2023-10-15 at...,0,0,576,1024,0.344767


In [37]:
model[0]['VGG-Face_cosine'][0]

0.11313706205403506

# Liveness check
img = frame[ymin:ymax, xmin:xmax]: This line extracts a region of interest (ROI) from the frame image. The ymin, ymax, xmin, and xmax values likely represent the coordinates of a bounding box around a detected face. This line of code creates a sub-image (img) by cropping the region of the frame within the specified bounding box.

img = cv2.resize(img, (32, 32)): After extracting the ROI, this line resizes the img to a smaller size of 32x32 pixels. It appears that the liveness detection model being used requires this specific input size.

img = img.astype('float') / 255.0: This line converts the pixel values in img to floating-point numbers and then normalizes them by dividing each pixel value by 255.0. Normalization is a common preprocessing step in machine learning to scale the pixel values from the original range (0-255 for RGB images) to a normalized range between 0 and 1. This helps the neural network converge faster during training.

img = tf.keras.preprocessing.image.img_to_array(img): This line converts the img from a NumPy array to a TensorFlow tensor. The img_to_array function is often used for preparing images for deep learning models.

img = numpy.expand_dims(img, axis=0): Finally, this line expands the dimensions of the img tensor by adding an extra dimension at the beginning. This is typically done to make the image compatible with models that expect a batch of images as input. By adding an extra dimension at the beginning, the shape of the img tensor becomes (1, 32, 32, 3) (assuming it's a color image with 3 channels).

In summary, the code is preparing a cropped and resized face image (img) for input to a liveness detection model. It resizes, normalizes, and converts the image to a format suitable for feeding into the model.







In [ ]:
!pip install xlwings

In [ ]:
!pip install pygame

In [ ]:
!pip install keyboard